<h1>Анализ формулы один с использованием методов машинного обучения и математической статистики</h1>

<h2>Первоначальной нашей целью будет прогноз вероятности совершения обгона на каждом из участков трассы.</h2>

In [1]:
import fastf1 as ff1
import pandas as pd
import numpy as np
import datetime
import math

ff1.Cache.enable_cache('cache')
session = ff1.get_session(2021, 22, 'R')
quali = ff1.get_session(2021, 22, 'Q')
quali = pd.DataFrame(quali.results)

In [2]:
laps = session.load_laps(with_telemetry=True)
session.load_telemetry()

core           INFO 	Loading laps for Abu Dhabi Grand Prix - Race [v2.1.13]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for session_status_data
utils          INFO 	NumExpr defaulting to 8 threads.
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 19 drivers: ['55', '7', '31', '33', '18', '77', '22', '14', '16', '44', '6', '63', '99', '3', '47', '11', '10', '4', '5']
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data


In [3]:
laps

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,SpeedST,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
0,0 days 01:03:54.885000,33,NaT,1,1,0 days 00:23:01.492000,NaT,NaT,0 days 00:00:38.428000,0 days 00:00:33.563000,...,318.0,SOFT,4.0,False,0 days 01:02:21.871000,Red Bull,VER,1,False,2021-12-12 13:03:21.881
1,0 days 01:05:23.988000,33,0 days 00:01:29.103000,2,1,NaT,NaT,0 days 00:00:18.116000,0 days 00:00:38.026000,0 days 00:00:32.961000,...,302.0,SOFT,5.0,False,0 days 01:03:54.885000,Red Bull,VER,1,True,2021-12-12 13:04:54.895
2,0 days 01:06:52.815000,33,0 days 00:01:28.827000,3,1,NaT,NaT,0 days 00:00:17.999000,0 days 00:00:38.043000,0 days 00:00:32.785000,...,301.0,SOFT,6.0,False,0 days 01:05:23.988000,Red Bull,VER,1,True,2021-12-12 13:06:23.998
3,0 days 01:08:21.841000,33,0 days 00:01:29.026000,4,1,NaT,NaT,0 days 00:00:18.019000,0 days 00:00:38.125000,0 days 00:00:32.882000,...,303.0,SOFT,7.0,False,0 days 01:06:52.815000,Red Bull,VER,1,True,2021-12-12 13:07:52.825
4,0 days 01:09:50.559000,33,0 days 00:01:28.718000,5,1,NaT,NaT,0 days 00:00:17.999000,0 days 00:00:38.083000,0 days 00:00:32.636000,...,303.0,SOFT,8.0,False,0 days 01:08:21.841000,Red Bull,VER,1,True,2021-12-12 13:09:21.851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,0 days 02:24:55.281000,47,0 days 00:02:11.605000,53,3,0 days 02:23:04.752000,NaT,0 days 00:00:40.395000,0 days 00:00:48.541000,0 days 00:00:42.669000,...,203.0,SOFT,4.0,False,0 days 02:22:43.676000,Haas F1 Team,MSC,4,False,2021-12-12 14:23:43.686
994,0 days 02:26:39.361000,47,0 days 00:01:44.080000,54,3,NaT,NaT,0 days 00:00:19.591000,0 days 00:00:42.890000,0 days 00:00:41.599000,...,268.0,SOFT,5.0,False,0 days 02:24:55.281000,Haas F1 Team,MSC,4,False,2021-12-12 14:25:55.291
995,0 days 02:28:57.976000,47,0 days 00:02:18.615000,55,3,NaT,NaT,0 days 00:00:22.400000,0 days 00:01:02.169000,0 days 00:00:54.046000,...,257.0,SOFT,6.0,False,0 days 02:26:39.361000,Haas F1 Team,MSC,4,False,2021-12-12 14:27:39.371
996,0 days 02:31:17.001000,47,0 days 00:02:19.025000,56,3,NaT,NaT,0 days 00:00:28.870000,0 days 00:00:56.316000,0 days 00:00:53.839000,...,258.0,SOFT,7.0,False,0 days 02:28:57.976000,Haas F1 Team,MSC,4,False,2021-12-12 14:29:57.986


Сделаем подготовку данных для просчета обгонов
Отберем для каждого гонщика те записи, где менялся гонщик впереди, аналогичные же записи отберем для других гонщиков

In [4]:
def get_peleton_changes():
    success_rate = 0
    fail_rate = 0
    drivers_peleton = list(map(int, reversed(quali['number'].tolist())))
    i = 0
    while True:
        try:
            driver_laps = laps.pick_driver(drivers_peleton[i]).get_telemetry()
            break
        except Exception as _:
            i += 1
    drivers_peleton = drivers_peleton[i:]
    driver_laps = driver_laps[(driver_laps['DriverAhead'] != '') & (driver_laps['Time'] > pd.Timedelta('0 days 00:00:05'))]
    # взять текущий тик -> ближайший тик гонщика впереди -> ближайший тик его гонщика и т.д. ...
    current_time = driver_laps.iloc[i]['Time']
    for _, tick in driver_laps.iterrows():
        driver_ahead_num = tick['DriverAhead']
        current_peleton = [int(drivers_peleton[0]), int(tick['DriverAhead'])]
        current_time_index = driver_laps[driver_laps['Time'] > current_time].index.min()
        current_time = driver_laps.loc[[current_time_index]].iloc[0]['Time']
        while driver_ahead_num != '':
            driver_ahead = laps.pick_driver(driver_ahead_num).get_telemetry()
            ahead_tick = driver_ahead[(driver_ahead['Time'] >= current_time)].index.min()
            ahead_tick = driver_ahead.loc[ahead_tick]
            driver_ahead_num = ahead_tick['DriverAhead']
            if driver_ahead_num != '':
                driver_ahead_num = int(driver_ahead_num)
                if driver_ahead_num not in current_peleton:
                    current_peleton.append(driver_ahead_num)
                elif driver_ahead_num == current_peleton[-2]:
                    current_peleton = current_peleton[:-2] + [current_peleton[-1], current_peleton[-2]]
                    driver_ahead = laps.pick_driver(driver_ahead_num).get_telemetry()
                    current_time_index = driver_ahead[(driver_ahead['Time'] > current_time) & (driver_ahead['DriverAhead'] != '')].index.min()
                    current_time = driver_ahead.loc[[current_time_index]].iloc[0]['Time']
                    print(current_peleton, 'peleton updated')
                else:
                    current_peleton = current_peleton[:2]
                    fail_rate += 1
                    print('Tick skipped', current_time)
                    break
            else:
                not_calculated_drivers = list(set(drivers_peleton).difference(current_peleton))
                if len(not_calculated_drivers) == 0:
                    success_rate += 1
                else:
                    fail_rate += 1
                current_peleton = current_peleton[:2]
                print('Tick ended', current_time)
                break
    print(success_rate / (success_rate + fail_rate))


get_peleton_changes()

Tick ended 0 days 00:00:05.300000
Tick ended 0 days 00:00:05.367000
Tick ended 0 days 00:00:05.520000
Tick ended 0 days 00:00:05.607000
Tick ended 0 days 00:00:05.740000
Tick ended 0 days 00:00:05.847000
Tick ended 0 days 00:00:05.960000
Tick ended 0 days 00:00:06.127000
Tick ended 0 days 00:00:06.180000
Tick ended 0 days 00:00:06.367000
Tick ended 0 days 00:00:06.400000
Tick ended 0 days 00:00:06.568000
Tick ended 0 days 00:00:06.620000
Tick ended 0 days 00:00:06.840000
Tick ended 0 days 00:00:06.848000
Tick ended 0 days 00:00:07.041000
Tick ended 0 days 00:00:07.088000
Tick ended 0 days 00:00:07.261000
Tick ended 0 days 00:00:07.328000
Tick ended 0 days 00:00:07.481000
[33, 47] peleton updated
Tick ended 0 days 00:00:07.701000
Tick skipped 0 days 00:00:07.808000
[4, 47] peleton updated
Tick ended 0 days 00:00:08.048000
Tick skipped 0 days 00:00:08.141000
Tick skipped 0 days 00:00:08.288000
Tick skipped 0 days 00:00:08.361000
Tick skipped 0 days 00:00:08.528000
Tick skipped 0 days 00:

KeyboardInterrupt: 

## Посмотрим на визуализацию мест обгонов на одной конкретной трассе за разные года. Для этого выберем трассу которая не менялась с 2014 года.
 <i style="color: #ff2800"> небольшое уточнение, все координаты представлены в 1/10 метра, мы разделим всю трассу на зоны

In [175]:
df = laps.pick_driver(44).get_telemetry()
df[df['Time'] > pd.Timedelta('0 days 00:00:42')]

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
369,2021-12-05 17:34:26.770,0 days 01:02:19.552000,,3.407942,0 days 00:00:42.038000,9058,145,4,64,0,1,car,2192.728333,0.007188,OnTrack,-4355,21782,123
370,2021-12-05 17:34:26.875,0 days 01:02:19.657000,,3.408469,0 days 00:00:42.143000,9244,146,4,66,0,1,pos,2197.014681,0.007202,OnTrack,-4396,21769,123
371,2021-12-05 17:34:27.010,0 days 01:02:19.792000,,3.408997,0 days 00:00:42.278000,9431,148,4,68,0,1,car,2202.595000,0.007220,OnTrack,-4446,21745,122
372,2021-12-05 17:34:27.095,0 days 01:02:19.877000,,3.409524,0 days 00:00:42.363000,9571,150,4,72,0,1,pos,2206.162137,0.007232,OnTrack,-4477,21728,122
373,2021-12-05 17:34:27.250,0 days 01:02:20.032000,,3.410051,0 days 00:00:42.518000,9711,153,4,77,0,1,car,2212.795000,0.007254,OnTrack,-4531,21690,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66112,2021-12-05 19:39:59.709,0 days 03:07:52.491000,,0.230000,0 days 02:06:14.977000,8799,248,8,0,100,0,car,305027.872222,0.999898,OnTrack,-1191,-70,118
66113,2021-12-05 19:39:59.720,0 days 03:07:52.502000,,0.230000,0 days 02:06:14.988000,8711,246,8,0,100,0,pos,305028.624336,0.999900,OnTrack,-1194,-64,118
66114,2021-12-05 19:39:59.939,0 days 03:07:52.721000,,0.230000,0 days 02:06:15.207000,8624,244,8,0,100,0,pos,305043.416787,0.999948,OnTrack,-1238,63,118
66115,2021-12-05 19:39:59.949,0 days 03:07:52.731000,,0.230000,0 days 02:06:15.217000,8537,243,8,0,100,0,car,305044.072222,0.999951,OnTrack,-1240,68,118


In [317]:
x_size_of_sector = 50
y_size_of_sector = 50

In [6]:
driver_numbers =  laps['DriverNumber'].unique()
overtakes = []
data_frame_to_process = {}
for driver in driver_numbers:
    telemetry = laps.pick_driver(driver).get_telemetry()
    df = telemetry['DriverAhead'].to_frame()
    df['DriverAhead'] = pd.to_numeric(df['DriverAhead'])
    df = df.fillna(0)
    df = df.diff(axis=0)
    telemetry['DriverAhead_difference'] = df['DriverAhead']
    telemetry.dropna(subset = ['DriverAhead_difference'], inplace=True)
    telemetry = telemetry[(telemetry['DriverAhead_difference'] != 0)]
    print(telemetry['DriverAhead_difference'].unique())
    # for _, overtake in telemetry.iterrows():
    #     drivers = sorted([int(overtake['DriverAhead']), int(driver)])
    #     overtakes.append({
    #         'First_driver': drivers[0],
    #         'Second_driver': drivers[1],
    #         'X': math.floor(overtake['X'] // 10 // x_size_of_sector),
    #         'Y': math.floor(overtake['Y'] // 10 // y_size_of_sector),
    #         'Date': overtake['Time'] - datetime.timedelta(microseconds=overtake['Time'].microseconds)
    #     })
overtakes = pd.DataFrame(overtakes)

[ 77. -63.  41. -45.   8.   4.  -4.  -8.  -9.  -1.  43. -43.  51.  22.
 -22. -39.  -2. -10.   7.  33. -33.   5.  28.  11. -11. -77.  25.  16.
 -41.  -7.  37.  44. -51. -44.]
[ 31. -31.   7.  -1.  93. -36. -49.   4.  -8.  85. -94.  58. -56.  -7.
  55. -41. -14.  11.  44. -44. -11. -55. -22.  22.]
[ 22.  55. -30. -31.  -9.  70. -74.  28.  46. -22. -83.  61. -61.  47.
 -52.  44.  -8.   8. -33. -19.  13.  -6.  81. -93.  41. -36. -11.  88.
 -96.   2.  -2.   3.  16.  39. -39.   6.  -3.  20. -21.  21. -26.  45.
 -57.  25. -13.  -7. -47. -44. -63.   7.  -5.  11.  -4.   4.   5.  17.
 -12. -45.  67. -67. -41. -55. -28. -15.   9.  31. -42.  42. -29.  29.
  33.]
[ 77. -22. -37.  -4.  41.   8. -32.  32.  -8. -17.  55. -55.  -6.  15.
 -21.   4.  85. -93.  57. -16.  -3. -11.  11.  22. -33.  25.  19. -13.
  13.  -5.   2.   9.  47. -45. -12.  -7. -10.  16.  -9.  28. -51.  51.
  18. -18.  10. -44.  44.]
[ 77. -66.   3.   4.  -4.  49.  36. -68. -27.  95. -95.  59. -59.  14.
 -13.   6.  66. -55. -18.  -1.

KeyboardInterrupt: 

In [319]:
overtakes = overtakes.drop_duplicates(subset=['First_driver', 'Second_driver', 'Date'])

<h2> Давайте создадим нашу трассу </h2>

In [320]:
def num_of_overtakes_in_segment(point, overtakes_df):
    return len(overtakes_df[(overtakes_df['X'] == math.floor(point['X'] // 10  // x_size_of_sector)) & (overtakes_df['Y'] == math.floor(point['Y'] // 10 // y_size_of_sector))])

lap = laps.pick_fastest()
tel = lap.get_telemetry()

x = np.array(tel['X'].values)
y = np.array(tel['Y'].values)

overtakes_to_display = pd.DataFrame({"X": x, "Y": y})
overtakes_to_display['nOvertakes'] = overtakes_to_display.apply(lambda i: num_of_overtakes_in_segment(i, overtakes), axis=1)

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
overtakes_to_display.sort_values(by='nOvertakes', ascending=False)

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

fig,ax = plt.subplots(figsize=(30, 30), dpi=200)

overtake = overtakes_to_display['nOvertakes'].to_numpy().astype(float)

cmap = cm.get_cmap('Paired')
lc_comp = LineCollection(segments, norm=plt.Normalize(1, cmap.N+1), cmap=cmap)
lc_comp.set_array(overtake)
lc_comp.set_linewidth(4)
ax.add_collection(lc_comp)

# plt.plot(x, y)
# plt.scatter(overtakes_to_display[overtakes_to_display['nOvertakes'] != 0]['X'], overtakes_to_display[overtakes_to_display['nOvertakes'] != 0]['Y'], color='y', s=20)
# plt.scatter(overtakes['X'] * 10 * x_size_of_sector, overtakes['Y'] * 10 * y_size_of_sector, color='r', s=10)
# plt.axis('equal')

major_ticks = np.arange(-7000, 23000, 500)
minor_ticks = np.arange(-7000, 23000, 500)

ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
plt.grid()
plt.xticks(rotation=90)
plt.show()